<h1 align="center" style="margin: 0; font-size: 36px;">Computational Intelligence for Optimization</h1>
<br>
<h1 align="center" style="margin: 0; font-size: 30px;">Music Festival Lineup Optimization</h1>

<br>

**Group members:**<br>
Bárbara Capitão - 20211532@novaims.unl.pt <br>
Carolina Silvestre - 20211512@novaims.unl.pt <br>
Francisco Pontes -  <br>
Lara Leandro - 20211632@novaims.unl.pt <br>


### Objective
Design the optimal festival lineup by assigning each artist to a specific stage and time slot, optimizing three key criteria:

1. **Prime Slot Popularity**  
   Maximize the total popularity of artists scheduled in the final time slot (prime slot) on each stage.

2. **Genre Diversity**  
   Maximize the diversity of musical genres across stages in each time slot. For every time slot, compute the average number of unique genres, normalized by the number of stages.

3. **Conflict Penalty**  
   Minimize fan base conflicts by avoiding scheduling artists with overlapping fan bases at the same time on different stages. This penalty is calculated using a provided pairwise conflict matrix.

---

### Constraints
- Each artist must be assigned to **exactly one stage and one time slot**.
- All stages must have **the same number of time slots**.
- All time slots occur **simultaneously** across all stages.
- No artist can be scheduled more than once or left unassigned.

---

### Representation
- The lineup is modeled as a 2D matrix:  
  `lineup[stage][slot] = artist_id`
- There are **5 stages and 7 time slots**, for a total of **35 artists**.

---



the fitness function
➔ at least 3 mutation operators
➔ at least 2 crossover operators
➔ at least 2 selection mechanisms

## Imports and Setup

In [1]:
import sys
sys.path.append(r'C:\Users\Bárbara C\CIFO-24-25\library')  

from copy import deepcopy
from algorithms.genetic_algorithms.algorithm import genetic_algorithm, get_best_ind
from algorithms.genetic_algorithms.crossover import standard_crossover, cycle_crossover
from algorithms.genetic_algorithms.mutation import binary_standard_mutation, swap_mutation, inversion_mutation
from solution import Solution


import utils

In [88]:
import numpy as np
import pandas as pd
import random
import itertools
from collections import defaultdict
from typing import Callable


random.seed(42)
np.random.seed(42)

## Load Data

In [3]:
artists = pd.read_csv('artists(in).csv')

In [4]:
conflict_matrix = pd.read_csv('conflicts(in).csv', index_col=0)

In [5]:
artists.rename(columns={'Unnamed: 0': 'artist_id'}, inplace=True)
artists.set_index('artist_id', inplace=True)

In [6]:
artists 

,name,genre,popularity
artist_id,,,
0,Midnight Echo,Rock,75
1,Solar Flare,Electronic,78
2,Velvet Pulse,Jazz,35
3,Neon Reverie,Electronic,100
4,The Silver Owls,Classical,85
5,Echo Chamber,Electronic,98
6,Aurora Skies,Pop,75
7,Static Mirage,Rock,94
8,Crimson Harmony,Classical,20


In [7]:
conflict_matrix

,Midnight Echo,Solar Flare,Velvet Pulse,Neon Reverie,The Silver Owls,Echo Chamber,Aurora Skies,Static Mirage,Crimson Harmony,Deep Resonance,...,Rhythm Alchemy,Cloud Nine Collective,Hypnotic Echoes,The Polyrhythm Syndicate,Harmonic Dissonance,Turbo Vortex,The Jazz Nomads,The Bassline Architects,Cosmic Frequency,Parallel Dimension
Midnight Echo,0.00,0.00,0.00,0.20,0.50,0.00,0.80,1.00,0.20,0.50,...,0.20,0.80,1.00,1.00,0.65,1.00,0.40,0.40,1.00,0.20
Solar Flare,0.00,0.00,0.00,1.00,0.00,1.00,0.00,1.00,0.00,0.65,...,0.20,0.65,0.00,0.65,0.40,0.40,0.40,0.00,0.00,1.00
Velvet Pulse,0.00,0.00,0.00,1.00,0.50,0.00,0.65,1.00,0.50,1.00,...,1.00,1.00,0.40,1.00,0.70,0.00,1.00,0.15,1.00,0.40
Neon Reverie,0.20,1.00,1.00,0.00,0.20,0.90,0.20,1.00,0.00,1.00,...,0.00,0.00,0.00,0.20,0.65,0.65,0.20,0.00,0.20,1.00
The Silver Owls,0.50,0.00,0.50,0.20,0.00,1.00,0.00,1.00,0.90,0.00,...,1.00,0.65,0.00,0.20,0.90,0.20,0.20,0.40,1.00,0.00
Echo Chamber,0.00,1.00,0.00,0.90,1.00,0.00,0.50,1.00,0.80,0.65,...,0.00,0.20,0.00,0.50,0.65,0.15,0.50,0.40,1.00,1.00
Aurora Skies,0.80,0.00,0.65,0.20,0.00,0.50,0.00,1.00,1.00,0.40,...,0.00,1.00,1.00,0.20,0.80,0.65,0.20,1.00,0.20,0.50
Static Mirage,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.20,0.20,...,1.00,0.80,0.90,0.65,0.00,1.00,0.40,0.50,1.00,0.20
Crimson Harmony,0.20,0.00,0.50,0.00,0.90,0.80,1.00,0.20,0.00,0.00,...,0.00,0.40,0.65,0.00,0.90,0.15,1.00,1.00,0.65,0.15
Deep Resonance,0.50,0.65,1.00,1.00,0.00,0.65,0.40,0.20,0.00,0.00,...,0.90,0.50,0.20,1.00,0.70,1.00,1.00,0.50,0.00,0.20


### Coherence Check 

In [8]:
num_artists = len(artists)
num_conflict_rows, num_conflict_cols = conflict_matrix.shape

# Ensure the number of artists matches the expected total
expected_artists = 5 * 7  # 5 stages x 7 slots
consistency_check = (num_artists == expected_artists and 
                     num_conflict_rows == expected_artists and 
                     num_conflict_cols == expected_artists)

# Ensure conflict matrix aligns with artist names
artist_names_match = list(artists['name']) == list(conflict_matrix.columns)

consistency_check, artist_names_match

(True, True)

### Generating Random Solution / Tests - Apagar

In [9]:
artist_indices = list(range(35))
random.shuffle(artist_indices)
lineup = np.array(artist_indices).reshape((5, 7))  

# Helper functions
def get_prime_slot_popularity(lineup):
    prime_artists = [lineup[stage][-1] for stage in range(5)]
    total_popularity = sum(artists.iloc[artist]['popularity'] for artist in prime_artists)
    max_possible = 5 * 100  # Max popularity (100) in each of 5 prime slots
    return total_popularity / max_possible

def get_genre_diversity_score(lineup):
    genres = artists['genre'].unique()
    max_unique = len(genres)
    total_score = 0

    for slot in range(7):
        genres_in_slot = set()
        for stage in range(5):
            artist_index = lineup[stage][slot]
            genres_in_slot.add(artists.iloc[artist_index]['genre'])
        total_score += len(genres_in_slot) / max_unique
    
    return total_score / 7  # Average over all slots

def get_conflict_penalty_score(lineup):
    worst_conflict = 0
    actual_conflict = 0

    for slot in range(7):
        artists_in_slot = [lineup[stage][slot] for stage in range(5)]
        for i in range(5):
            for j in range(i + 1, 5):
                a_i = artists.iloc[artists_in_slot[i]]['name']
                a_j = artists.iloc[artists_in_slot[j]]['name']
                conflict = conflict_matrix.at[a_i, a_j]
                actual_conflict += conflict
                worst_conflict += 1  # worst case: all conflicts = 1

    normalized_conflict = actual_conflict / worst_conflict if worst_conflict else 0
    return 1 - normalized_conflict  # because conflict is a penalty
    
lineup_tuple = (
    (3, 17, 23, 34, 31, 16,  9),
    (25,  2, 24, 10, 30, 15, 26),
    (22, 14, 18,  6,  7, 28,  5),
    (29, 19, 11, 32, 27, 13, 21),
    (33, 12,  4,  0,  8,  1, 20)
)

# Compute scores
prime_score = get_prime_slot_popularity(lineup_tuple)
genre_score = get_genre_diversity_score(lineup_tuple)
conflict_score = get_conflict_penalty_score(lineup_tuple)

# Final fitness (average of normalized scores)
fitness = (prime_score + genre_score + conflict_score) / 3

lineup_tuple, {
    'Prime Slot Popularity': round(float(prime_score),4),
    'Genre Diversity': round(float(genre_score),4),
    'Conflict Penalty': round(float(conflict_score),4),
    'Final Fitness': round(float(fitness),4)
}


(((3, 17, 23, 34, 31, 16, 9),
  (25, 2, 24, 10, 30, 15, 26),
  (22, 14, 18, 6, 7, 28, 5),
  (29, 19, 11, 32, 27, 13, 21),
  (33, 12, 4, 0, 8, 1, 20)),
 {'Prime Slot Popularity': 0.936,
  'Genre Diversity': 0.7381,
  'Conflict Penalty': 0.6193,
  'Final Fitness': 0.7645})

In [76]:
import random

artist_indices = list(range(35))
random.shuffle(artist_indices)

# Create a list of lists with the desired dimensions (5 rows, 7 columns)
lineup = [artist_indices[i*7 : (i+1)*7] for i in range(5)]

print(lineup)

[[18, 27, 3, 6, 25, 11, 29], [9, 33, 15, 26, 31, 1, 22], [30, 14, 10, 4, 12, 7, 17], [13, 2, 20, 16, 5, 8, 32], [28, 24, 19, 34, 23, 21, 0]]


In [77]:
lineup

[[18, 27, 3, 6, 25, 11, 29],
 [9, 33, 15, 26, 31, 1, 22],
 [30, 14, 10, 4, 12, 7, 17],
 [13, 2, 20, 16, 5, 8, 32],
 [28, 24, 19, 34, 23, 21, 0]]

In [11]:
swap_mutation(lineup, 0.5)

array([[ 9, 12,  5, 20, 31, 22, 29],
       [ 8, 19, 30,  6, 25,  0, 33],
       [16, 26, 24, 14, 10, 11, 27],
       [13, 18,  2, 32, 28, 21,  3],
       [23,  4, 34, 15, 17,  1,  7]])

In [41]:
 block_move_mutation_matrix(lineup, 1)

[[9, 12, 5, 6, 25, 0, 33],
 [13, 18, 2, 32, 28, 21, 3],
 [20, 31, 22, 29, 16, 26, 24],
 [14, 10, 11, 27, 8, 19, 30],
 [23, 4, 34, 15, 17, 1, 7]]

### Artist Lineup Solution

In [33]:

class ArtistLineupSolution(Solution):
    def __init__(self, conflict_matrix, artists, repr=None):
        self.conflict_matrix = conflict_matrix
        self.artists = artists

        if repr is None:
            repr = self.random_initial_representation()
        else:
            self._validate_repr(repr)

        super().__init__(repr=repr)

    def random_initial_representation(self):
        artist_indices = list(range(35))
        random.shuffle(artist_indices)
        # 5 stages × 7 slots
        return [artist_indices[i:i + 7] for i in range(0, 35, 7)]

    def _validate_repr(self, repr):
        # Must be 5 rows and 7 columns, with unique values from 0 to 34
        if not isinstance(repr, list):
            raise TypeError("Representation must be a list of lists")

        if len(repr) != 5 or not all(len(row) == 7 for row in repr):
            raise ValueError("Representation must be 5 rows of 7 integers")

        flat = [a for row in repr for a in row]

        if set(flat) != set(range(35)):
            raise ValueError("Representation must contain all 35 artist indices exactly once")

    def fitness(self):
        def get_prime_slot_popularity():
            # Last slot in each stage is the prime slot
            prime_artists = [self.repr[stage][6] for stage in range(5)]
            total_popularity = sum(self.artists.iloc[artist]['popularity'] for artist in prime_artists)
            return total_popularity / (5 * 100)

        def get_genre_diversity_score():
            genres = self.artists['genre'].unique()
            total_score = 0
            for slot in range(7):
                genres_in_slot = set()
                for stage in range(5):
                    artist_index = self.repr[stage][slot]
                    genres_in_slot.add(self.artists.iloc[artist_index]['genre'])
                total_score += len(genres_in_slot) / len(genres)
            return total_score / 7

        def get_conflict_penalty_score():
            actual_conflict = 0
            worst_conflict = 0
            for slot in range(7):
                artists_in_slot = [self.repr[stage][slot] for stage in range(5)]
                for i in range(5):
                    for j in range(i + 1, 5):
                        a_i = self.artists.iloc[artists_in_slot[i]]['name']
                        a_j = self.artists.iloc[artists_in_slot[j]]['name']
                        actual_conflict += self.conflict_matrix.at[a_i, a_j]
                        worst_conflict += 1
            return 1 - (actual_conflict / worst_conflict if worst_conflict else 0)

        return (get_prime_slot_popularity() + get_genre_diversity_score() + get_conflict_penalty_score()) / 3


In [34]:
class ArtistLineupSolutionGA(ArtistLineupSolution):
    def __init__(
        self,
        conflict_matrix,
        artists,
        mutation_function,
        crossover_function,
        repr=None
    ):
        self.mutation_function = mutation_function
        self.crossover_function = crossover_function
        super().__init__(conflict_matrix, artists, repr)

    def crossover(self, other_solution):
        offspring1_repr, offspring2_repr = self.crossover_function(self.repr, other_solution.repr)
        return (
            ArtistLineupSolutionGA(
                conflict_matrix=self.conflict_matrix,
                artists=self.artists,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring1_repr
            ),
            ArtistLineupSolutionGA(
                conflict_matrix=self.conflict_matrix,
                artists=self.artists,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring2_repr
            )
        )

    def mutation(self, mut_prob):
        new_repr = self.mutation_function(self.repr, mut_prob)
        return ArtistLineupSolutionGA(
            conflict_matrix=self.conflict_matrix,
            artists=self.artists,
            mutation_function=self.mutation_function,
            crossover_function=self.crossover_function,
            repr=new_repr
        )


### Mutation Functions

In [81]:
def block_move_mutation_matrix(lineup, mut_prob):
    """
    Applies block move mutation to a lineup matrix.

    Block move mutation selects a random contiguous block of artists, 
    removes it, and reinserts it at another valid position, ensuring no duplication 
    and preserving the 5x7 shape.

    Parameters:
    ----------
    lineup : list of list of int
        A 5x7 matrix representing the lineup. Each artist ID must be unique.
    mut_prob : float
        Probability of applying the mutation (between 0 and 1).

    Returns:
    -------
    list of list of int
        A mutated 5x7 matrix with a moved block, or a copy of the original.
    """
    if random.random() <= mut_prob:

        # Flatten matrix to 1D permutation
        flat = [artist for stage in lineup for artist in stage]
    
        # Select random block
        start = random.randint(0, len(flat) - 2)
        end = random.randint(start + 1, len(flat))  # at least size 1
    
        block = flat[start:end]
        remainder = flat[:start] + flat[end:]
    
        # Choose insertion index in remainder
        insert_pos = random.randint(0, len(remainder))
        new_flat = remainder[:insert_pos] + block + remainder[insert_pos:]
    
        # Reshape to 5x7 matrix
        new_lineup = [
            new_flat[i * 7:(i + 1) * 7] for i in range(5)
        ]
    
        return new_lineup
        
    else:
        return deepcopy(lineup)

        


In [82]:
def scramble_mutation_matrix(lineup, mut_prob):
    """
    Applies scramble mutation to a lineup matrix.

    Scramble mutation randomly selects a rectangular block (possibly from multiple stages and slots),
    flattens and shuffles it, then reinserts it in the same location. The size and position of the block 
    vary randomly. Ensures all artists are preserved and only reordered within the block.

    Parameters:
    ----------
    lineup : list of list of int
        2D matrix representing the lineup. Must be 5x7, with unique artist IDs.
    mut_prob : float
        Probability of applying the mutation (between 0 and 1).

    Returns:
    -------
    list of list of int
        A new 5x7 lineup matrix with a scrambled sub-block, or a copy if mutation does not occur.
    """

    if random.random() <= mut_prob:
        # Choose block dimensions
        start_stage = random.randint(0, 4)
        end_stage = random.randint(start_stage, 4)

        start_slot = random.randint(0, 6)
        end_slot = random.randint(start_slot, 6)

        # Extract and flatten the sub-block
        sub_block = [
            new_lineup[s][t]
            for s in range(start_stage, end_stage + 1)
            for t in range(start_slot, end_slot + 1)
        ]
        random.shuffle(sub_block)

        # Reinsert shuffled values
        idx = 0
        for s in range(start_stage, end_stage + 1):
            for t in range(start_slot, end_slot + 1):
                new_lineup[s][t] = sub_block[idx]
                idx += 1

        return new_lineup
        
    else:
        return deepcopy(lineup)




In [83]:
def conflict_reduction_mutation(lineup, mut_prob):
    """
    Applies a conflict-aware mutation to reduce fan base overlap in the lineup.

    Parameters:
        lineup (list of lists or np.ndarray): 5x7 structure representing the current lineup,
            where lineup[stage][slot] = artist_id.
        mut_prob (float): The probability of applying the mutation.

    Returns:
        list of lists: A new 5x7 lineup after mutation (or a copy if no mutation occurs).
    """
    # Ensure we work with a NumPy array internally for easy slicing
    lineup = np.array(lineup)

    if random.random() <= mut_prob:
        new_lineup = deepcopy(lineup)

        worst_slot = -1
        worst_score = -1

        # Find time slot with highest conflict
        for slot in range(7):
            conflict = 0
            for i in range(5):
                for j in range(i + 1, 5):
                    a = new_lineup[i, slot]
                    b = new_lineup[j, slot]
                    conflict += conflict_matrix.iat[a, b]

            if conflict > worst_score:
                worst_score = conflict
                worst_slot = slot

        if worst_slot != -1:
            source_stage = random.randint(0, 4)
            target_slot = random.randint(0, 6)
            while target_slot == worst_slot:
                target_slot = random.randint(0, 6)

            # Swap artists
            new_lineup[source_stage, worst_slot], new_lineup[source_stage, target_slot] = (
                new_lineup[source_stage, target_slot],
                new_lineup[source_stage, worst_slot],
            )

        return new_lineup.tolist()  # Convert to list of lists before returning
    else:
        return deepcopy(lineup).tolist()  # Convert even in no-mutation case


### Crossover Functions

In [84]:
def row_wise_crossover(parent1_repr, parent2_repr):
    """
    Performs a row-wise crossover (swap stages) with conflict-free repair.

    Each child starts as a copy of a parent and randomly swaps full rows.
    Afterward, duplicates are fixed and missing artist IDs are inserted.

    Args:
        parent1_repr (list of lists): 5x7 matrix, valid permutation of artist IDs.
        parent2_repr (list of lists): 5x7 matrix, valid permutation of artist IDs.

    Returns:
        tuple: Two valid 5x7 offspring (list of lists), each using artist IDs 0–34 once.
    """
    def flatten(lineup):
        return [artist for row in lineup for artist in row]

    def to_matrix(flat):
        return [flat[i*7:(i+1)*7] for i in range(5)]

    # Start with copies
    child1 = [row.copy() for row in parent1_repr]
    child2 = [row.copy() for row in parent2_repr]

    # Randomly choose rows (stages) to swap
    rows_to_swap = random.sample(range(5), k=random.randint(1, 5))
    for row in rows_to_swap:
        child1[row], child2[row] = child2[row], child1[row]

    # Function to repair duplicates/missing values
    def repair(child):
        flat = flatten(child)
        seen = set()
        duplicates = []
        for i, artist in enumerate(flat):
            if artist in seen:
                duplicates.append(i)
            else:
                seen.add(artist)
        missing = list(set(range(35)) - seen)
        random.shuffle(missing)
        for i, idx in enumerate(duplicates):
            flat[idx] = missing[i]
        return to_matrix(flat)

    return repair(child1), repair(child2)


In [85]:
def partially_mapped_crossover(parent1_repr, parent2_repr):
    """
    Partially Mapped Crossover (PMX) for permutations.

    Swaps a middle segment between parents and maps conflicting values 
    to preserve a valid permutation.

    Args:
        parent1_repr (list): Parent 1 permutation.
        parent2_repr (list): Parent 2 permutation.

    Returns:
        tuple: Two valid offspring permutations.
    """
    size = len(parent1_repr)
    p1, p2 = parent1_repr[:], parent2_repr[:]

    # Choose crossover points
    cx_point1 = random.randint(0, size - 2)
    cx_point2 = random.randint(cx_point1 + 1, size - 1)

    def pmx(p1, p2):
        child = [None] * size
        # Copy middle slice
        child[cx_point1:cx_point2] = p1[cx_point1:cx_point2]

        # Map the rest from p2
        for i in range(cx_point1, cx_point2):
            if p2[i] not in child:
                val = p2[i]
                pos = i
                while True:
                    val_in_p1 = p1[pos]
                    pos = p2.index(val_in_p1)
                    if child[pos] is None:
                        child[pos] = val
                        break
        # Fill remaining positions
        for i in range(size):
            if child[i] is None:
                child[i] = p2[i]
        return child

    return pmx(p1, p2), pmx(p2, p1)


### Selection Functions

In [90]:
def tournament_selection(population, maximization=False, size=3):
    # Randomly selects 3 individuals from population
    selected = random.sample(population, size)
    
    if maximization:
        # If is a maximization problem selects the individual with the highest fitness
        best = max(selected, key=lambda ind: ind.fitness())
        
    else:
        # If is a minimization problem selects the individual with the smallest fitness
        best = min(selected, key=lambda ind: ind.fitness())
        
    return deepcopy(best)


In [91]:
def rank_based_selection(population,  maximization = True):
    # Sort individuals by fitness
    
    # If maximization, highest fitness gets rank 1 (best)
    # If minimization, lowest fitness gets rank 1 (best)
    sorted_pop = sorted(population, key=lambda ind: ind.fitness(), reverse=maximization)

    # Assign ranks: best individual gets highest rank (len), worst gets 1
    # This ensures the best individual has the highest chance to be selected
    ranks = list(range(len(sorted_pop), 0, -1))  
    total_rank = sum(ranks)  # Sum of all ranks, used to normalize selection probability

    # Generate a random number between 0 and total of ranks
    pick = random.uniform(0, total_rank)

    # Iterate over the population and sum up ranks
    current = 0
    for ind, rank in zip(sorted_pop, ranks):
        current += rank
        if current >= pick:
            # Once the random number falls in the "rank box", select the individual
            return deepcopy(ind)


### Genetic Algorithm

In [64]:
POP_SIZE = 50
population = [
    ArtistLineupSolutionGA(
        crossover_function=row_wise_crossover,
        mutation_function=conflict_reduction_mutation,
        conflict_matrix = conflict_matrix,
        artists = artists
        
    )
    for _ in range(POP_SIZE)
]

In [73]:
best_solution = genetic_algorithm(
    initial_population=population,
    max_gen=2,
    selection_algorithm=rank_based_selection,
    maximization=True,
    xo_prob=0.8,
    mut_prob=0.2,
    elitism=True,
    verbose=False
)

print("Best solution fitness", best_solution.fitness())

Best solution fitness 0.7162698412698413


## Impact of hyperparameters

In [96]:
POP_SIZE = 100
GENERATIONS = 200

grid_params = {
    "crossover": [
        {
            "function": row_wise_crossover,
            "mut_prob": 0.8
        },
        {
            "function": partially_mapped_crossover,
            "mut_prob": 0.8
        }
    ],
    "mutation": [
        {
            "function": block_move_mutation_matrix,
            "xo_prob": 0.2
        },
        {
            "function": scramble_mutation_matrix,
            "xo_prob": 0.2
        },
        {
            "function": conflict_reduction_mutation,
            "xo_prob": 0.2
        }
    ],
    "elitism": [True, False]
}

fitness_dfs = {}

# Generate all combinations of the grid parameters
grid = list(itertools.product(grid_params["crossover"], grid_params["mutation"], grid_params["elitism"]))

for crossover_cfg, mutation_cfg, elitism in grid:
    # Create empty dataframe for each configuration
    # Columns have the fitness in each generation, rows will have results for each run
    df = pd.DataFrame(columns=range(GENERATIONS)) # Shape will be 30 x 200

    # Get crossover and mutation functions and probabilities from the grid
    crossover_function = crossover_cfg.get("function")
    xo_prob = crossover_cfg.get("xo_prob", 0.8)  # Default fallback if not present
    
    mutation_function = mutation_cfg.get("function")
    mut_prob = mutation_cfg.get("mut_prob", 0.2)  # Default fallback


    config_results = []

    # Run for 30 times
    for run_nr in range(2):
        _, fitness_over_gens = genetic_algorithm(
            initial_population=[
                 ArtistLineupSolutionGA(
                    crossover_function=crossover_function,
                    mutation_function=mutation_function,
                    conflict_matrix = conflict_matrix,
                    artists = artists
                ) for _ in range(POP_SIZE)
            ],
            max_gen=GENERATIONS,
            selection_algorithm=tournament_selection,
            xo_prob=xo_prob,
            mut_prob=mut_prob,
            elitism=elitism
        )
        config_results.append(fitness_over_gens)

        df.loc[run_nr] = fitness_over_gens

    # Create a label for the configuration
    config_label = (
        f"{crossover_function.__name__}{xo_prob}_"
        f"{mutation_function.__name__}{mut_prob}_elitism={elitism}"
    )

    # Save configuration results in the dictionary
    fitness_dfs[config_label] = df

KeyboardInterrupt: 

In [ ]:
def plot_fitness_over_gen(fitness_dfs: dict[str, pd.DataFrame]):
    """
    
    """
    fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
    handles, labels = [], []

    for config_name, df in fitness_dfs.items():
        mean_fitness = df.mean(axis=0)
        median_fitness = df.median(axis=0)
        
        line1, = axes[0].plot(mean_fitness.index, mean_fitness.values, label=config_name)
        axes[1].plot(median_fitness.index, median_fitness.values, label=config_name)

        handles.append(line1)
        labels.append(config_name)

    axes[0].set_title("Mean Fitness Across Generations")
    axes[1].set_title("Median Fitness Across Generations")

    for ax in axes:
        ax.set_xlabel("Generation")
        ax.set_ylabel("Fitness")
        ax.grid(True)

    # Shared boxed legend below
    legend = fig.legend(
        handles,
        labels,
        loc='lower center',
        bbox_to_anchor=(0.5, -0.15),
        ncol=2,
        frameon=True,
        borderpad=1
    )

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.25)
    plt.show()

In [ ]:
plot_fitness_over_gen(fitness_dfs)